# subjects and parents
idea: create a tree (or multiple) of subjects and sub-subjects.  
use parent and child relationships as sequential relationships.

In [1]:
import psycopg2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# setup connection
conn = psycopg2.connect(
    host='localhost',
    database='cadets_e3',
    user='rosendahl',
)
conn.set_session(readonly=True)
# get cursor
cur = conn.cursor()

In [3]:
# load number of events, subjects, principals
no_events = 41350895
no_subjects = 224629
no_principals = 22

In [4]:
# debug
os.system("hostnamectl hostname")
print(f'current working directory: {os.getcwd()}')

vmrosendahl
current working directory: /home/rosendahl/remote_interpreter/dataset/jupyter


In [5]:
# load subject_blacklist
subject_blacklist = set()
with open('data/blacklisted_subjects_attack.txt', 'r') as file:
    for line in file:
        subject_blacklist.add(line.strip())

In [120]:
from functools import lru_cache

class Subject:
    uuid: str|None
    parent_uuid: str
    children: list
        
    def __init__(self, uuid: str, parent_uuid: str|None):
        self.uuid = uuid
        self.parent_uuid = parent_uuid
        self.children = list()
    
    def add_child(self, child):
        self.children.append(child)
    
    @lru_cache(maxsize=8096)
    def find_child(self, uuid):
        if self.uuid == uuid:
            return self
        for child in self.children:
            result = child.find_child(uuid)
            if result is not None:
                return result
        return None
        


In [121]:
# load subjects
query = '''
SELECT uuid, parentsubject_uuid
FROM subject;
'''

cur.execute(query)

orphan_count = 0

subjects = []
for row in cur:
    uuid, parent_uuid = row
    #if uuid in subject_blacklist:
    #    continue
    if parent_uuid is None:
        orphan_count += 1

    subjects.append(Subject(uuid, parent_uuid))

print(f'number of subjects: {len(subjects)}')
print(f'number of orphans: {orphan_count}')

root = Subject(None, None)

number of subjects: 224629
number of orphans: 849


In [122]:
print(f'number of subjects left: {len(subjects)}')

uuids_in_tree = set()
uuids_in_tree.add(None)

iteration = 0
while len(subjects) > 0:
    iteration += 1
    print(f'iteration: {iteration}, subjects left: {len(subjects)}')
    to_remove = []
    for subject in subjects:
        if subject.parent_uuid in uuids_in_tree:
            parent = root.find_child(subject.parent_uuid)
            parent.add_child(subject)
            to_remove.append(subject)
            uuids_in_tree.add(subject.uuid)
    if len(to_remove) == 0 and len(subjects) > 0:
        print('no more subjects can be added to the tree')
        raise Exception('no more subjects can be added to the tree')
    subjects = [subject for subject in subjects if subject not in to_remove]


number of subjects left: 224629
iteration: 1, subjects left: 224629
iteration: 2, subjects left: 147910
iteration: 3, subjects left: 22


In [128]:
# get tree statistics

max_depth = 0
uuid_deepest = None
max_children = 0
uuid_max_children = None
avg_depth = 0

def get_tree_statistics(node, depth):
    global max_depth
    global max_children
    global uuid_deepest
    global uuid_max_children
    global avg_depth
    if depth > max_depth:
        max_depth = depth
        uuid_deepest = node.uuid
    
    if len(node.children) == 0:
        avg_depth += depth   
    if len(node.children) > max_children:
        max_children = len(node.children)
        uuid_max_children = node.uuid
    for child in node.children:
        get_tree_statistics(child, depth + 1)

get_tree_statistics(root, 0)
avg_depth /= len(uuids_in_tree)

print(f'max depth: {max_depth} uuid: {uuid_deepest}')
print(f'max children: {max_children} uuid: {uuid_max_children}')
print(f'average depth: {avg_depth}')

max depth: 15 uuid: 989C31BF-3A10-11E8-B8CE-15D78AC88FB6
max children: 71974 uuid: 65C53022-39C4-11E8-B8CE-15D78AC88FB6
average depth: 2.9920625027823533
